In [1]:
import torch
import torch.nn as nn
from torch.nn import functional
from torch.nn import Module
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [2]:
import pickle
PATH = "poem_dict.obj"

In [63]:

with open(PATH,'rb') as f:
    data = pickle.load(f)
    allkeys = [key for key in data.keys()]
    input_data = []
    for tag in allkeys: 
        for poem in data[tag]:
            p = poem.lower().split()
            pp = " ".join(p)
            input_data.append(pp)

In [4]:
import string

all_characters = string.ascii_letters+ "unk" + " .,;'-" 
n_characters = len(all_characters)

In [107]:
print(all_characters)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZunk .,;'-


In [5]:
import random

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        letter = string[c]
        if letter not in all_characters:
            letter = 'unk'
        tensor[c] = all_characters.index(letter)
        
    return tensor
    

In [106]:
 def get_batch(chunk_len):
    poem_num = random.randint(0, len(input_data))
    text_str = input_data[poem_num][:chunk_len+1]

    text_input = char_tensor(text_str[:-1])
    text_target = char_tensor(text_str[1:])
    return text_input, text_target

In [108]:
x,y=get_batch(250)
x.shape,y.shape

(torch.Size([250]), torch.Size([250]))

In [131]:
class PoemLSTM(nn.Module):     
    """ 
    """
    def __init__(self, input_size, hidden_size, output_size, n_layers=1): 

        super(PoemLSTM, self).__init__()   
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        emb = self.embed(x)
#         print('emb out',emb.shape)
        
        lstm_in = emb.view(1, 1, -1)
#         print(lstm_in.shape)
#         print('in dim',lstm_in.dim())

        
        lstm_out, hn_cn = self.lstm(lstm_in)
#         print('lst_out shape',lstm_out.shape)
        fc_out = self.fc(lstm_out.view(1, -1))
#         print('fc_out',fc_out.shape)
        return fc_out, hn_cn

In [132]:
# Initialize Model 

hidden_size = 100
n_layers = 1
lr = 0.001


PoemGenerator = PoemLSTM(n_characters, hidden_size, n_characters, n_layers)

# Set Hyperparameter Values

NUM_EPOCHS = 100 
LEARNING_RATE = 0.001
OPTIMIZER = optim.Adam          
LOSS_FUNCTION = nn.functional.cross_entropy 

In [127]:
def train_model(model, optimizer_type, loss_function, learning_rate):
    """
    """

    model.train()

    optimizer = optimizer_type(params=model.parameters(), lr=learning_rate)        
    loss = 0

    input_sent,target_sent = get_batch(250)

    for c in range(250):
        print(input_sent[c].shape)
        output, _ = model(input_sent[c])
        print(output.shape, target_sent[c].shape)
        loss += loss_function(output, target_sent[c])

    # zero grad
    optimizer.zero_grad()

    loss.backward()

    # update parameters
    optimizer.step()

    print(f"Training loss: {loss.item()/250}")
    return loss.item()/250  
         

In [128]:
def sample(model, start_letter = 'A',predict_len=100,temperature = 0.7):
    
    with torch.no_grad():
        initial_input = char_tensor(start_letter)
        output_text = start_letter
        
        print(initial_input[0])
        
        _, (hidden, cell) = model(initial_input[0].view(1))
        inp_char = initial_input[-1]
        
        for p in range(predict_len):
            output, (hidden, cell) = model(inp_char.view(1))
            
            # Sample
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]
            
            # Add predicted character to string
            predicted_char = all_characters[top_char]
            output_text += predicted_char
            inp_char = char_tensor(predicted_char)

    return output_text

In [134]:
# train and sample
print_every = 50
loss_avg = 0
for epoch in range(NUM_EPOCHS):
    
    loss = train_model(model=PoemGenerator, optimizer_type=OPTIMIZER, loss_function=LOSS_FUNCTION, learning_rate=LEARNING_RATE)       
    loss_avg += loss
    
    if epoch % print_every == 0:
        print('epoch',epoch)
        print("Sample:",sample(model=PoemGenerator,start_letter = 'A'), '\n')

torch.Size([])
fc_out torch.Size([1, 61])
torch.Size([1, 61]) torch.Size([])


ValueError: Expected input batch_size (1) to match target batch_size (0).